## Create Database and Tables

In [0]:
CREATE DATABASE sales_performances_dev;
SET search_path TO public;

### Create Dimension Tables

In [0]:
CREATE TABLE stores (
    store_id INT PRIMARY KEY,
    store_name VARCHAR(255),
    store_city VARCHAR(255),
    store_address VARCHAR(255)
)
DISTSTYLE ALL;

CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    name VARCHAR(255),
    role VARCHAR(100),
    hire_date DATE,
    salary DECIMAL(10, 2)
)
DISTKEY(employee_id);

CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    category VARCHAR(100)
)
DISTSTYLE ALL;

### Create Fact Table

In [0]:
CREATE TABLE sales (
    employee_id INT PRIMARY KEY,
    sale_id INT,
    store_id INT,
    product_id INT,
    sale_date DATE,
    quantity INT,
    unit_price INT,
    total_amount DECIMAL(10, 2),
    FOREIGN KEY (store_id) REFERENCES stores(store_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
DISTKEY (employee_id)
SORTKEY (sale_date)
;

## Load Data & Create View

### Load Data into Tables

In [ ]:
COPY employees
    FROM 's3://redshift-project/employees.csv' 
    DELIMITER ','
    CSV
    DATEFORMAT 'YYYY-MM-DD' 
    REGION 'eu-west-2'
    IGNOREHEADER 1
    IAM_ROLE 'arn:aws:iam::291064975419:role/service-role/AmazonRedshift-CommandsAccessRole-20241207T104511';
    
COPY products
    FROM 's3://redshift-project/products.csv' 
    DELIMITER ',' 
    CSV
    DATEFORMAT 'YYYY-MM-DD' 
    REGION 'eu-west-2'
    IGNOREHEADER 1
    IAM_ROLE 'arn:aws:iam::291064975419:role/service-role/AmazonRedshift-CommandsAccessRole-20241207T104511';

COPY stores
    FROM 's3://redshift-project/stores.csv' 
    DELIMITER ','
    CSV 
    DATEFORMAT 'YYYY-MM-DD' 
    REGION 'eu-west-2'
    IGNOREHEADER 1
    IAM_ROLE 'arn:aws:iam::291064975419:role/service-role/AmazonRedshift-CommandsAccessRole-20241207T104511';

COPY sales(sale_id,store_id,employee_id,product_id,sale_date,quantity,unit_price,total_amount)
    FROM 's3://redshift-project/sales.csv'
    DELIMITER ','
    CSV 
    DATEFORMAT 'YYYY-MM-DD' 
    REGION 'eu-west-2'
    IGNOREHEADER 1
    IAM_ROLE 'arn:aws:iam::291064975419:role/service-role/AmazonRedshift-CommandsAccessRole-20241207T104511';

### Create View

In [ ]:
CREATE VIEW employee_monthly_sales AS(
    with employee_monthly_sales_by_id as(
        select 
            sales.employee_id,
            DATE_TRUNC('month', sales.sale_date) AS month,
            sum(sales.total_amount) as total_sales
        FROM
            sales
        WHERE
            DATE_PART('year', sales.sale_date) = 2024
        GROUP BY
            sales.employee_id, month
        ORDER BY
            sales.employee_id, sum(sales.total_amount) DESC
    )

    SELECT
        e.employee_id,
        e.name,
        e.role,
        em.month,
        em.total_sales
    from 
        employees as e
        left join employee_monthly_sales_by_id as em
            on e.employee_id = em.employee_id
)

## Create User and Grant Access For Tableau

In [ ]:
CREATE USER tableau_user PASSWORD 'your-password';
GRANT SELECT ON public.employee_monthly_sales TO tableau_user;